In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from matplotlib import pyplot as plt 

import warnings
warnings.filterwarnings("ignore")

In [2]:
DATAPATH = "../input/megaGymDataset_cleaned.csv"

# Load the dataset
dataset = pd.read_csv(DATAPATH)
dataset.fillna('', inplace=True)
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


In [3]:
def recommend_exercises(user_input):
    data = pd.read_csv(DATAPATH)
    
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])
    
    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Build the ANN model using TensorFlow
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(len(data["ID"].unique()), activation='softmax')  # Use the unique number of IDs as the output dimension
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

In [4]:
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input)
dataset.iloc[IDs]



Epoch 1/10


73/73 [==============================] - 1s 7ms/step - loss: 8.0056 - accuracy: 0.0000e+00 - val_loss: 8.0179 - val_accuracy: 0.0000e+00
Epoch 2/10
73/73 [==============================] - 0s 5ms/step - loss: 7.8381 - accuracy: 0.0030 - val_loss: 8.5072 - val_accuracy: 0.0000e+00
Epoch 3/10
73/73 [==============================] - 0s 5ms/step - loss: 7.0581 - accuracy: 0.0021 - val_loss: 10.0717 - val_accuracy: 0.0000e+00
Epoch 4/10
73/73 [==============================] - 0s 5ms/step - loss: 6.2616 - accuracy: 0.0043 - val_loss: 12.4430 - val_accuracy: 0.0000e+00
Epoch 5/10
73/73 [==============================] - 0s 5ms/step - loss: 5.8237 - accuracy: 0.0103 - val_loss: 13.8851 - val_accuracy: 0.0000e+00
Epoch 6/10
73/73 [==============================] - 0s 5ms/step - loss: 5.5256 - accuracy: 0.0086 - val_loss: 15.4649 - val_accuracy: 0.0000e+00
Epoch 7/10
73/73 [==============================] - 0s 5ms/step - loss: 5.3074 - accuracy: 0.0120 - val_loss: 16.6898 - val_

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
855,855,UN Pull-Up Supinated,,Strength,Biceps,Body Only,Intermediate,9.000000
854,854,Pull-up - Gethin Variation,The pull-up is a multi-joint bodyweight exerci...,Strength,Biceps,Body Only,Intermediate,9.000000
853,853,TBS Chin-Up,The a chin-up is a variation of the pull-up ex...,Strength,Biceps,Body Only,Intermediate,9.000000
857,857,UN Pull-Up Neutral,,Strength,Biceps,Body Only,Intermediate,9.000000
97,97,Decline bar press sit-up,The decline bar press sit-up is a weighted cor...,Strength,Abdominals,Barbell,Intermediate,8.500000
17,17,30 Barbell Roll-Out,The barbell roll-out is an abdominal exercise ...,Strength,Abdominals,Barbell,Intermediate,7.750000
909,909,Standing Calf Raise,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833
910,910,Standing calf raise-,The standing calf raise is a popular movement ...,Strength,Calves,Body Only,Intermediate,6.870833
911,911,HM Left Calf-Spike Sumo Pulse,,Strength,Calves,Body Only,Intermediate,6.870833
697,697,Side-lying leg adduction scissor,The side-lying leg adduction scissor is a dyna...,Strength,Adductors,Body Only,Intermediate,4.816667


In [5]:
IDs

[855, 854, 853, 857, 97, 17, 909, 910, 911, 697]